<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)"> Predictive Analysis - Image Processing</h2>

[source](https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/)

#### Classify handwritten digits using the famous MNIST data

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/mnist.png)

The goal in this task is to take an image of a handwritten single digit, and determine what that digit is.  

The data for this competition were taken from the MNIST dataset. The MNIST ("Modified National Institute of Standards and Technology") dataset is a classic within the Machine Learning community that has been extensively studied.  More detail about the dataset, including Machine Learning algorithms that have been tried on it and their levels of success, can be found at http://yann.lecun.com/exdb/mnist/index.html.  

In [1]:
import os
import sys
import time
import glob
import math
import random
import pickle
import pylab
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot

from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.decomposition import PCA

import tensorflow as tf

%matplotlib inline
#matplotlib.rcdefaults()
#matplotlib.verbose.set_level('silent')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from IPython.display import Image
from IPython.core.display import HTML

In [2]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-02-19 10:35:36.238686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 10:35:36.266496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 10:35:36.266640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.python.keras import utils
from sklearn.metrics import accuracy_score

In [4]:
mnist = tf.keras.datasets.mnist

# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# let's print the shape of the dataset

In [5]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (60000, 28, 28)
y_train shape (60000,)
X_test shape (10000, 28, 28)
y_test shape (10000,)


In [6]:
# Flattening the images from the 28x28 pixels to 1D 787 pixels
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [7]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [8]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = tf.keras.utils.to_categorical(y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


## ANN

In [9]:
# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(100, input_shape=(784,), activation='relu'))
model.add(Dense(10, activation='softmax'))

# looking at the model summary
model.summary()

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
%time model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


2022-02-19 10:35:36.562369: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-19 10:35:36.563196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 10:35:36.563401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 10:35:36.563517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3786 - accuracy: 0.8971 - val_loss: 0.2065 - val_accuracy: 0.9423
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1776 - accuracy: 0.9495 - val_loss: 0.1476 - val_accuracy: 0.9570
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1299 - accuracy: 0.9633 - val_loss: 0.1211 - val_accuracy: 0.9645
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1030 - accuracy: 0.9704 - val_loss: 0.1041 - val_accuracy: 0.9688
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0847 - accuracy: 0.9754 - val_loss: 0.0945 - val_accuracy: 0.9720
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0716 - accuracy: 0.9791 - val_loss: 0.0858 - val_accuracy: 0.9745
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0601 - accuracy: 0.9826 - val_loss: 0.0803 - val_accuracy: 0.9751
Epoch 

In [10]:
model.evaluate(X_test, Y_test, verbose=1)

313/313 [==============================] - 0s 1ms/step - loss: 0.0788 - accuracy: 0.9772


[0.07881936430931091, 0.9771999716758728]

## CNN

In [11]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [12]:
# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [13]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [14]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = tf.keras.utils.to_categorical(y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


In [15]:
# building a linear stack of layers with the sequential model
model = Sequential()

model.add(Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

# looking at the model summary
model.summary()

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


# training the model for 10 epochs
%time model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 10)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6760)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               676100    
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 677,210
Trainable params: 677,210
Non-trainable params: 0
________________________________________________

2022-02-19 10:35:46.506778: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302


 20/469 [>.............................] - ETA: 1s - loss: 1.3218 - accuracy: 0.6879  

2022-02-19 10:35:46.841262: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


469/469 [==============================] - 2s 3ms/step - loss: 0.2247 - accuracy: 0.9381 - val_loss: 0.0803 - val_accuracy: 0.9756
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0699 - accuracy: 0.9792 - val_loss: 0.0586 - val_accuracy: 0.9805
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0459 - accuracy: 0.9866 - val_loss: 0.0571 - val_accuracy: 0.9821
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0301 - accuracy: 0.9910 - val_loss: 0.0507 - val_accuracy: 0.9833
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0205 - accuracy: 0.9940 - val_loss: 0.0519 - val_accuracy: 0.9840
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0138 - accuracy: 0.9961 - val_loss: 0.0572 - val_accuracy: 0.9834
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0104 - accuracy: 0.9971 - val_loss: 0.0573 - val_accuracy: 0.9834
Epoch 8/10
469/46

In [16]:
model.evaluate(X_test, Y_test, verbose=1)

313/313 [==============================] - 0s 1ms/step - loss: 0.0691 - accuracy: 0.9827


[0.0691300705075264, 0.982699990272522]